In [10]:
# importando dependências
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score    
from tpot import TPOTRegressor as TPR
import sherpa
import keras
from keras.models import Sequential
from keras.layers import LSTM, Flatten, Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor

In [11]:
# Carregando Dataset
def dataset():
    import pandas_datareader.data as web
    import datetime as dt

    end = dt.datetime(2020, 11, 1)
    start = dt.datetime(2015, 1, 1)

    df = web.DataReader("AAPL", 'yahoo', start, end)

    df = df.reset_index()
    df = df.drop(columns=['Open','Date','High','Low','Volume','Adj Close'])
    df = df.rename(columns={'Close': 'Close 0'})

    def window (df, w):
        for i in range(1,w):
            df['Close '+str(i)] = df['Close '+str(i-1)].shift(1)
        return df

    new_df = window(df,5)
    new_df = new_df.rename(columns={'Close 0': 'Target'})
    new_df.dropna(inplace=True)

    limit = int(len(new_df)*0.75)

    X = new_df.loc[:,['Close 1','Close 2','Close 3','Close 4']]
    y = new_df.loc[:,'Target'].tolist()

    X = np.array(X)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    y = np.array(y)

    X_train, X_test = X[:limit], X[limit:]
    y_train, y_test = y[:limit], y[limit:]
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = dataset()

In [12]:
#Função para selecionar nós
def find_nodes(n_layers, n_first, n_last):
    layers = []
    increment = (n_last - n_first) / (n_layers - 1)
    nodes = n_first
    for i in range (1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + increment

    return layers

#Criando modelo base

def create_model(optimizer='adam', activation = 'relu', dropout = 0.0, hidden = 1, n_first = 50, n_last = 5):
    
    model = Sequential()
    model.add(Flatten())
    nodes = find_nodes(hidden, n_first, n_last)

    for i in range(0, hidden-1):
        if i == 0:
            model.add(Dense(n_first, input_dim=X_train.shape[1], activation = activation))
        else:
            model.add(Dense(nodes[i], activation = activation))
            model.add(Dropout(dropout))
    
    model.add(Dense(1))
    
    model.compile(loss='mse', optimizer=optimizer, 
                  metrics=[keras.metrics.MeanSquaredError()])
    return model

In [13]:
# batch_size e epochs
batch_size = [10, 20, 60, 100]
epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
dropout_rate = [0.0, 0.1, 0.2, 0.3]
node_first = [30, 50, 75]
node_last = [5, 7, 10]
hidden = [2,3,4]

In [36]:
#Criando dicionario de parametros

model_config = {'tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor':{
    'batch_size':[10, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':['SGD', 'Adam', 'Adamax'],
    'activation':['relu', 'tanh', 'sigmoid',],
    'dropout_rate':[0.0, 0.1, 0.2, 0.3],
    'node_first':[30, 50, 75],
    'node_last':[5, 7, 10],
    'hidden':[2,3,4]
}}

In [25]:
#Usando keras wrapper para transformar rede em um modelo
modelo = KerasRegressor(build_fn = create_model, verbose=1)

In [35]:
print(modelo)

In [37]:
model_tpot = TPR(generations= 5, population_size= 30, random_state=1, config_dict=model_config, cv= 5,
                 verbosity = 2, scoring='r2')

In [38]:
tpot_result = model_tpot.fit(X_test, y_test)

Version 0.11.6.post2 of tpot is outdated. Version 0.11.7 was released Wednesday January 06, 2021.


ValueError: Error: Input data is not in a valid format. Please confirm that the input data is scikit-learn compatible. For example, the features must be a 2-D array and target labels must be a 1-D array.

In [40]:
y_train

array([27.97249985, 28.00250053, 27.3125    , ..., 47.52000046,
       47.25      , 45.77249908])